<a href="https://colab.research.google.com/github/crcifuentesc/parcial-2-A/blob/main/Dashboards_Parcial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guía Streamlit en Colab y Ngrok - Dashboards

Elaborado por: Juan David Muñoz Buritica jmunozbu@unal.edu.co (Monitoria SyS 2024-1)
Revisado: Andrés Marino Álvarez Meza amalvarezme@unal.edu.co

- [Streamlit](https://streamlit.io/) es una librería destinada a crear y compartir aplicaciones o dashboards.
- Su objetivo es ser una herramienta fácil de usar que permita ejecutar scripts directamente y desplegarlos en un aplicativo web.
- Sin embargo, antes de ver su funcionamiento, debemos tener en cuenta que, si bien Streamlit permite correr el aplicativo, debemos alojarlo y "hacerlo visible" a la red.
- Existen diferentes métodos que cumplen dicho objetivo, también dependiendo del nivel de madurez que se requiera, bien sea simplemente testeo o producción. - Una alternativa eficaz, rápida y que puede ser fácilmente escalable es [Ngrok](https://ngrok.com/).

- Ngrok es un servicio y aplicativo globalmente distribuido que asegura, acelera y protege las aplicaciones y servicios de red.
- En este caso, nos servirá para crear el tunel http que nos deje visualizar el dashboard corriendo con Streamlit desde el entorno del cuadernillo o script de Python.
- El servicio gratuito de Ngrok, tras crear la cuenta tiene la posibilidad de mantener un agente activo simultáneo, cuya configuración está dada por el token de autenticación único por usuario y túnel.

**Nota: Cree una cuenta en Ngrok https://ngrok.com/ con su correo UNAL y guarde el token personal asignado**

In [1]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.9 MB/s eta 0:00:00


In [2]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile #librerias descarga Youtube y manejo de audios en python

     | 2.7 MB 5.4 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.8-py3-none-any.whl.metadata (9.9 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.6 MB/s eta 0:00:00
Using cached charse

In [3]:
# Instalar ffmpeg si es necesario en Colab
!apt-get install ffmpeg

import os
import streamlit as st
from yt_dlp import YoutubeDL

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


La estructura básica para correr un programa o script en streamlit es la siguiente.

- A continuación se muestra una app básica con algunas de las funcionalidades básicas para visualizar en dashboard.

- Debemos crear un archivo .py con los códigos del dashboard para su posterior visualización en Ngrok.

In [4]:
%%writefile detector.py

import streamlit as st
import numpy as np
import os
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from pyngrok import ngrok
import yt_dlp as youtube_dl
import joblib
import warnings
import scipy.signal as sig


warnings.filterwarnings("ignore")

# Configuración de la señal y bases
A = 1  # Amplitud de la señal
T = 5  # Duración en segundos
Fs = 100  # Frecuencia de muestreo
F_o = 1 / T  # Frecuencia fundamental
N = 50  # Número de armónicos
tv = np.arange(-T/2, T/2, 1/Fs)  # Vector de tiempo generado
Nm = len(tv)
phin = np.zeros((Nm, 2 * N + 1), dtype=np.complex_)
xo = A**2 * np.sin(2 * np.pi * F_o * tv)**2
wo = 2*np.pi/T
for n in range(-N, N+1, 1):
    phin[:,n+N] = np.exp(1j*n*wo*tv)

Cn = np.zeros(2 * N + 1, dtype=np.complex_)
nv = np.linspace(-N, N, 2 * N + 1)
Cn[N] = A**2 / 2
Cn[N + 2] = -A**2 / 4
Cn[N - 2] = -A**2 / 4
Px = A**2


# Función para graficar las bases
def pltbase(n=1):
    fig, ax = plt.subplots(1, 1, figsize=(6, 3))
    ax.plot(tv, np.real(phin[:, int(n + N)]), label="Re{$\phi_n$(t)}")
    ax.plot(tv, np.imag(phin[:, int(n + N)]), label="Im{$\phi_n$(t)}")
    ax.plot(tv, xo, 'r', linewidth=4, label='$x(t)$')
    ax.set_xlabel("t[s]", fontsize=14)
    ax.set_ylabel("$\phi_n$(t)", fontsize=14)
    ax.grid(True)
    ax.legend(loc='upper left')
    st.pyplot(fig)

# Función para graficar el espectro
def plot_spectrum(Cn, nv):
    fig = plt.figure(figsize=(8, 8))
    plt.subplot(2, 2, 1)
    plt.stem(nv, np.real(Cn), 'r', use_line_collection=True)
    plt.xlabel(r'$n$', fontsize=14)
    plt.ylabel(r'$Re\{C_n\}$', fontsize=14)
    plt.grid(True)
    plt.subplot(2, 2, 2)
    plt.stem(nv, np.imag(Cn), 'r', use_line_collection=True)
    plt.xlabel(r'$n$', fontsize=14)
    plt.ylabel(r'$Im\{C_n\}$', fontsize=14)
    plt.grid(True)
    plt.subplot(2, 2, 3)
    plt.stem(nv, np.abs(Cn), 'r', use_line_collection=True)
    plt.xlabel(r'$n$', fontsize=14)
    plt.ylabel(r'$|C_n|$', fontsize=14)
    plt.grid(True)
    plt.subplot(2, 2, 4)
    plt.stem(nv, np.angle(Cn), 'r', use_line_collection=True)
    plt.xlabel(r'$n$', fontsize=14)
    plt.ylabel(r'$\langle C_n$', fontsize=14)
    plt.grid(True)
    fig.tight_layout()
    return fig

# Función para calcular y graficar la reconstrucción de la señal
def pltest(Na=1):
    ind = np.arange(N - Na, N + Na + 1)
    er = 1 - np.sum(np.abs(Cn[ind])**2) / Px
    xe = phin[:, ind].dot(Cn[ind])
    fig, ax = plt.subplots()
    ax.plot(tv, np.real(xe), color='b', label='$x_e(t)$ (Señal estimada)')
    ax.plot(tv, xo, color='r', label='$x(t)$ (Señal original)')
    ax.set_title(f'$E_r$ = {100 * er:.2f}%', fontsize=16)
    ax.set_xlabel('t[s]')
    ax.set_ylabel('x(t)')
    ax.grid(True)
    ax.legend()
    st.pyplot(fig)


@st.cache_data
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    video_info = youtube_dl.YoutubeDL().extract_info(url = video_url,download=False)
    filename = f"{name}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    print("Download complete... {}".format(filename))

@st.cache_resource
def load_and_process_audio_files(path='results/'):
    # Function to train the model from the Excel file
    pass


# widgets y funcionalidades de Streamlit
st.title("Parcial#2")
st.write("### 1. Espectro de Fourier de la señal $ x(t) = |A \sin(2\pi F_o t)|^2 $, Dado que la señal está definida en  $[-1/2Fo, 1/2Fo]$, con  $ A, F_o \in \mathbb{R^+} $:")

# Widget interactivo en Streamlit para cambiar n
n = st.slider('Elige el armónico n', min_value=-N, max_value=N, value=0, step=1)
pltbase(n)

st.write('2. Realice las simulaciones respectivas para graficar el espectro de Fourier del ejercicio 1 (magnitud y fase como diagrama de Bode en decibelios), y presente el error relativo y la senal reconstruida para  N = {${1, 2, . . . , 50}$}.')

# Mostrar la gráfica del espectro en Streamlit
fig = plot_spectrum(Cn, nv)
st.pyplot(fig)

# Slider interactivo en Streamlit para seleccionar Na
Na = st.slider('Elige el número de armónicos Na', min_value=1, max_value=N, value=1, step=1)
pltest(Na)




# Sección de modulación AM
st.write('Sea la señal portadora $c(t) = A_c\sin(2\pi F_c t),$ con $A_c,F_c\in\mathbb{R},$ y la señal mensaje $m(t)\in\mathbb{R}$. Encuentre el espectro en frecuencia de la señal modulada en amplitud (AM), $ y(t) = \left(1+\frac{m(t)}{A_c}\right)c(t) $.')
custom_url = st.text_input("Enter a new YouTube URL to process:", "https://www.youtube.com/watch?v=aZiQ7aJ_BVo")


import os
import streamlit as st
from yt_dlp import YoutubeDL
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np

# Función para descargar el video de YouTube como MP3
@st.cache_data
def download_ytvid_as_mp3(video_url, name):
    video_info = YoutubeDL().extract_info(url=video_url, download=False)
    filename = f"{name}.mp3"
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': filename,
    }

    with YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    return filename

# Función para convertir de MP3 a WAV usando ffmpeg
def convert_mp3_to_wav(mp3_file, wav_file):
    os.system(f"ffmpeg -i {mp3_file} {wav_file}")

# Función principal para procesar el archivo WAV
def process_wav_file(nombre_out, ti, tf, fs, selection):
    xa, fs = sf.read(nombre_out)  # Leer archivo WAV
    st.write(f'Frecuencia de muestreo: {fs:.2f} Hz\nArchivo de audio: {nombre_out}')

    xpro = xa.copy()  # Copiar archivo para procesar
    m = xpro[int(ti * fs):int(tf * fs), 0]  # Señal mensaje en el primer canal (5 segundos)

    # Reproducir el fragmento de audio
    st.audio(nombre_out, format='audio/wav')

    # Llamar a la función correcta según la selección del usuario
    if selection == 'Portadora':
        plot_carrier_signal(m, fs, ti, tf)
    elif selection == 'Modulada':
        plot_modulated_signal(m, fs, ti, tf)

# Función para procesar y graficar la portadora
def plot_carrier_signal(m, fs, ti, tf):
    tt = np.arange(0, tf - ti, 1 / fs)  # Vector de tiempo
    Fc = 15000  # Frecuencia portadora
    Im = 0.5  # Índice de modulación
    Ac = max(abs(m)) / Im  # Amplitud de la portadora
    c = Ac * np.sin(2 * np.pi * Fc * tt)  # Señal portadora

    # Fourier de la portadora
    Cw = np.fft.rfft(c)
    vf = np.fft.rfftfreq(len(c), d=1/fs)  # Frecuencias asociadas a la FFT

    # Graficar magnitud del espectro de la portadora
    fig, ax = plt.subplots()
    ax.plot(vf, abs(Cw))
    ax.set_title(r'Espectro Portadora')
    ax.set_xlabel(r'$f[Hz]$', fontsize=14)
    ax.set_ylabel(r'$|C[f]|$', fontsize=14)
    st.pyplot(fig)

    # Graficar fase del espectro de la portadora
    fig, ax = plt.subplots()
    ax.plot(vf, np.angle(Cw))
    ax.set_xlabel(r'$f[Hz]$', fontsize=14)
    ax.set_ylabel(r'$<C[f]$', fontsize=14)
    st.pyplot(fig)

# Función para procesar y graficar la señal modulada en AM
def plot_modulated_signal(m, fs, ti, tf):
    tt = np.arange(0, tf - ti, 1 / fs)  # Vector de tiempo
    Fc = 15000  # Frecuencia portadora
    Im = 0.5  # Índice de modulación
    Ac = max(abs(m)) / Im  # Amplitud de la portadora
    c = Ac * np.sin(2 * np.pi * Fc * tt)  # Señal portadora

    # Señal modulada
    y = (1 + m / Ac) * c  # Señal modulada en AM

    # Fourier de la señal modulada
    Yw = np.fft.rfft(y)
    vf = np.fft.rfftfreq(len(y), d=1/fs)  # Frecuencias asociadas a la FFT

    # Graficar magnitud del espectro de la señal modulada
    fig, ax = plt.subplots()
    ax.plot(vf, abs(Yw))
    ax.set_title(r'Espectro Señal Modulada')
    ax.set_xlabel(r'$f[Hz]$', fontsize=14)
    ax.set_ylabel(r'$|Y[f]|$', fontsize=14)
    st.pyplot(fig)

    # Graficar fase del espectro de la señal modulada
    fig, ax = plt.subplots()
    ax.plot(vf, np.angle(Yw))
    ax.set_xlabel(r'$f[Hz]$', fontsize=14)
    ax.set_ylabel(r'$<Y[f]$', fontsize=14)
    st.pyplot(fig)


# Interfaz de Streamlit
st.title("Procesamiento de Audio - Descarga y Conversión")

# Input del enlace de YouTube
custom_url = st.text_input("Ingrese la URL de YouTube para procesar:", "https://www.youtube.com/watch?v=aZiQ7aJ_BVo")
# Selector entre Portadora y Modulación
selection = st.radio("Seleccione el tipo de señal para graficar:", ('Portadora', 'Modulada'))

if st.button("Descargar y procesar audio"):
    # Paso 1: Descargar el audio como MP3
    mp3_filename = download_ytvid_as_mp3(custom_url, "audio")

    # Paso 2: Convertir el archivo MP3 a WAV
    wav_filename = "output.wav"
    convert_mp3_to_wav(mp3_filename, wav_filename)

    # Paso 3: Procesar y graficar la señal seleccionada
    process_wav_file(wav_filename, ti=20, tf=25, fs=44100, selection=selection)





# Parámetros de simulación
Fo = 60  # Frecuencia de alimentación en Hz
Fs = 30 * Fo  # Frecuencia de muestreo
To = 1 / Fo  # Periodo fundamental
Ts = 1 / Fs  # Periodo de muestreo
t = np.arange(0, 5 * To, Ts)  # Simulación de 5 periodos

A = 120  # Amplitud de la entrada

# Onda senoidal rectificada
in_o = A * np.abs(np.sin(2 * np.pi * Fo * t))  # Onda rectificada completa

# Función de transferencia del circuito RC serie
def transferencia_RC(R, C):
    num = [1]
    den = [R * C, 1]
    return sig.TransferFunction(num, den)

# Simulación para carga resistiva pura (RC -> R grande, C pequeño)
def simular_salida(R, C, in_o, t):
    G_n = transferencia_RC(R, C)
    _, out, _ = sig.lsim(G_n, in_o, t)
    return out

# Espectro de frecuencia con FFT
def calcular_fft(signal, Ts):
    vfre = np.fft.rfftfreq(len(signal), Ts)  # Vector de frecuencias
    Xf = np.fft.rfft(signal)  # FFT de la señal
    return vfre, Xf

# Cálculo del THD
def calcular_thd(Xf):
    V1 = np.abs(Xf[1]) / len(Xf)  # Componente fundamental
    Vh = np.sqrt(np.sum(np.abs(Xf[2:])**2)) / len(Xf)  # Suma de los armónicos
    THD = Vh / V1
    return THD

# Función principal que toma R y C como entrada
def simulacion(R_v_res, C_v_res, R_v_RC, C_v_RC):
    # Simulación de salida para ambos casos
    out_res = simular_salida(R_v_res, C_v_res, in_o, t)  # Carga resistiva
    out_RC = simular_salida(R_v_RC, C_v_RC, in_o, t)  # Carga RC serie

    # Cálculo del espectro de ambas salidas
    vfre_res, Xf_res = calcular_fft(out_res, Ts)
    vfre_RC, Xf_RC = calcular_fft(out_RC, Ts)

    # Gráfica del espectro de frecuencias para el caso resistivo
    st.write("Espectro de Frecuencia - Carga Resistiva")
    fig, ax = plt.subplots()
    ax.stem(vfre_res, np.abs(Xf_res) / len(out_res), basefmt=" ", use_line_collection=True)
    ax.set_xlabel('Frecuencia (Hz)')
    ax.set_ylabel('Magnitud')
    ax.grid(True)
    st.pyplot(fig)

    # Gráfica del espectro de frecuencias para el caso RC serie
    st.write("Espectro de Frecuencia - Carga RC Serie")
    fig, ax = plt.subplots()
    ax.stem(vfre_RC, np.abs(Xf_RC) / len(out_RC), basefmt=" ", use_line_collection=True)
    ax.set_xlabel('Frecuencia (Hz)')
    ax.set_ylabel('Magnitud')
    ax.grid(True)
    st.pyplot(fig)

    # Cálculo del THD para ambas cargas
    THD_res = calcular_thd(Xf_res)
    THD_RC = calcular_thd(Xf_RC)

    st.write(f'THD (Carga Resistiva) = {THD_res * 100:.2f}%')
    st.write(f'THD (Carga RC Serie) = {THD_RC * 100:.2f}%')

# Entradas de usuario para los valores de R y C
st.title("Simulación de Circuitos RC")

R_v_res = st.number_input("Ingrese el valor de la resistencia (R) para el caso resistivo (ohmios):", min_value=1, value=1000)
C_v_res = st.number_input("Ingrese el valor de la capacitancia (C) para el caso resistivo (faradios):", min_value=0.0, value=1e-12)

R_v_RC = st.number_input("Ingrese el valor de la resistencia (R) para el circuito RC (ohmios):", min_value=1, value=1000)
C_v_RC = st.number_input("Ingrese el valor de la capacitancia (C) para el circuito RC (faradios):", min_value=0.0, value=10e-6)

if st.button("Ejecutar simulación"):
    simulacion(R_v_res, C_v_res, R_v_RC, C_v_RC)



Writing detector.py


Como al momento de correr el script con Streamlit se requiere del archivo .py donde se encuentre el código, utilizamos %%writefile para crearlo. Posteriormente hacemos toda la configuración de Ngrok con el token y demás datos de protocolo.

In [5]:
token = '2loZ1IszAq0Ffqh3A6XzAHrPxEr_6Qnh1WtYf76TARtMwsZAG' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [8]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run detector.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://c740-34-125-250-87.ngrok-free.app
